In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [2]:
# Load the training data
train_data = pd.read_csv('/Users/admin-20218/Downloads/house-prices-advanced-regression-techniques/train.csv')

# Load the test data
test_data = pd.read_csv('/Users/admin-20218/Downloads/house-prices-advanced-regression-techniques/test.csv')

# Load the sample submission file
sample_submission = pd.read_csv('/Users/admin-20218/Downloads/house-prices-advanced-regression-techniques/sample_submission.csv')

In [3]:
# Print the first few rows of the dataset to verify that it loaded correctly
print(train_data.head(10))

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN      Reg   

  LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1         Lvl    AllPub  ...       

In [4]:
print(test_data.head(10))

     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   
5  1466          60       RL         75.0    10000   Pave   NaN      IR1   
6  1467          20       RL          NaN     7980   Pave   NaN      IR1   
7  1468          60       RL         63.0     8402   Pave   NaN      IR1   
8  1469          20       RL         85.0    10176   Pave   NaN      Reg   
9  1470          20       RL         70.0     8400   Pave   NaN      Reg   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub  ...         120        0    NaN  MnPrv         NaN   
1    

In [5]:
print(sample_submission.head(10))

     Id      SalePrice
0  1461  169277.052498
1  1462  187758.393989
2  1463  183583.683570
3  1464  179317.477511
4  1465  150730.079977
5  1466  177150.989247
6  1467  172070.659229
7  1468  175110.956520
8  1469  162011.698832
9  1470  160726.247831


In [6]:
scaler = StandardScaler()
scaled_sample_submission = scaler.fit_transform(sample_submission)

In [7]:
# Data setup
from sklearn.model_selection import train_test_split

X_train = train_data.drop(["Id", "SalePrice"], axis=1)
y_train = train_data["SalePrice"]
X_test = test_data.drop("Id", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [8]:
# Feature selection
continuous_features = ["LotArea", "GrLivArea"]
categorical_features = ["MSZoning", "Neighborhood"]



In [9]:
# Feature processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), continuous_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)



In [10]:
# Model training
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_processed, y_train)



LinearRegression()

In [11]:
# Model evaluation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_squared_log_error



def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

y_pred = model.predict(X_val_processed)
rmse = sqrt(mean_squared_error(y_val, y_pred))
rmsle = compute_rmsle(np.log(y_val), np.log(y_pred))

print("RMSE:", rmse)
print("RMSLE:", rmsle)


RMSE: 42645.392558882624
RMSLE: 0.02


In [12]:
import sklearn
print(sklearn.__version__)

1.2.2
